In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
#import geopandas as gpd
#import folium
#from shapely.geometry import point
#from folium.plugins import MarkerCluster


%matplotlib inline

In [ ]:
#pd.set_option('display.max_rows', 10)

## Imported and appended all excel files and corresponding worksheets

In [ ]:
file = pd.ExcelFile('../Data/overall_statepop_clean.xlsx')
names = file.sheet_names
overall_statepop = pd.concat([file.parse(name) for name in names])
overall_statepop = overall_statepop.reset_index(drop=True)

In [ ]:
file = pd.ExcelFile('../Data/overall_cocpop_clean.xlsx')
names = file.sheet_names
overall_cocpop = pd.concat([file.parse(name) for name in names])
overall_cocpop = overall_cocpop.reset_index(drop=True)

In [ ]:
file = pd.ExcelFile('../Data/category_state.xlsx')
names = file.sheet_names
state_category = pd.concat([file.parse(name) for name in names])
state_category = state_category.reset_index(drop=True)
state_category

In [ ]:
file = pd.ExcelFile('../Data/category_cocpop.xlsx')
names = file.sheet_names
coc_category = pd.concat([file.parse(name) for name in names])
coc_category = coc_category.reset_index(drop=True)

In [ ]:
file = pd.ExcelFile('../Data/beds_state.xlsx')
names = file.sheet_names
state_beds = pd.concat([file.parse(name) for name in names])
state_beds = state_beds.reset_index(drop=True)

In [ ]:
file = pd.ExcelFile('../Data/coc_beds.xlsx')
names = file.sheet_names
coc_beds = pd.concat([file.parse(name) for name in names])
coc_beds = coc_beds.reset_index(drop=True)

In [ ]:
overall_statepop

## Census API for loop 

In [ ]:
#from census import Census
#from us import states

#c = Census('API Key')

#census_df = c.acs.get(('NAME', 'B01001_001E'), {'for': 'state:*'}, year=2009)
#census_df = pd.DataFrame(census_df)
#census_df = census_df.rename(columns={'B01001_001E':'2009'}).drop(columns='state')

#loop_df = pd.DataFrame()
#loop_dict = {}
#for i in range(2010,2021):
#    loop_dict = c.acs.get(('NAME', 'B01001_001E'), {'for': 'state:*'}, year=i)
#    loop_df = pd.DataFrame(loop_dict)
#    loop_df = loop_df.rename(columns={'B01001_001E':str(i)}).drop(columns ='state')
#    loop_df = loop_df.loc[loop_df.NAME != 'Puerto Rico'].reset_index(drop=True)
#    census_df = census_df.merge(loop_df, on = 'NAME')

#census_df.head()

In [ ]:
#census_df.to_excel('census_data.xlsx', sheet_name = 'census')

In [ ]:
census_df = pd.read_excel('../Data/census_data.xlsx')
census_df.drop(columns='Unnamed: 0', inplace = True)
census_df.rename(columns = {'NAME':'State'}, inplace = True)
census_df.head()

## For loop for percent of homeless overall 

In [ ]:
test = []
test1 = []
test2 = []
test3 = []
test4 = []
test5 = []
test6 = []
for i in range(2009,2021):
    test.append((overall_statepop['Overall Homeless'].loc[overall_statepop['Year'] == i].sum() / census_df[str(i)].sum()) * 100)
    test1.append(i)
    test2.append(((overall_statepop['Sheltered Total Homeless'].loc[overall_statepop['Year'] == i].sum() / overall_statepop['Overall Homeless'].loc[overall_statepop['Year'] == i].sum()) * 100))
    test3.append(((overall_statepop['Unsheltered Total Homeless'].loc[overall_statepop['Year'] == i].sum() / overall_statepop['Overall Homeless'].loc[overall_statepop['Year'] == i].sum()) * 100))
    test4.append(((state_category['Overall Homeless Veterans'].loc[state_category['Year'] == i].sum() / state_category['Overall Homeless'].loc[state_category['Year'] == i].sum()) * 100))
    test5.append(((state_category['Overall Homeless Youth'].loc[state_category['Year'] == i].sum() / state_category['Overall Homeless'].loc[state_category['Year'] == i].sum()) * 100))
    test6.append(((state_category['Overall Homeless People in Families'].loc[state_category['Year'] == i].sum() / state_category['Overall Homeless'].loc[state_category['Year'] == i].sum()) * 100))
year_perc = pd.DataFrame()
year_perc['Year'] = test1
year_perc['Percent_Overall_Pop'] = test
year_perc['Sheltered_Perc'] = test2
year_perc['Unsheltered_Perc'] = test3
year_perc['Veteran_Perc'] = test4
year_perc['Youth_Perc'] = test5
year_perc['Families_Perc'] = test6
year_perc

## Calculating beds per homeless overall

In [ ]:
beds_pop = pd.merge(overall_statepop, state_beds)
beds_pop['Beds_Per_Homeless'] = beds_pop['Total Year-Round Beds'] / beds_pop['Overall Homeless']
beds_pop

## Calculating beds per homeless categories by state

In [ ]:
beds_per_category = pd.merge(state_category, state_beds)
beds_per_category['Beds_Veteran'] = beds_per_category['Dedicated Veteran Beds'] / beds_per_category['Overall Homeless Veterans']
beds_per_category['Beds_Youth'] = beds_per_category['Dedicated Youth Beds'] / beds_per_category['Overall Homeless Veterans']
beds_per_category['Beds_Families'] = beds_per_category['Total Beds for Households with Children'] / beds_per_category['Overall Homeless Veterans']
beds_per_category['Beds_Families_NoKids'] = beds_per_category['Total Beds for Households without Children'] / beds_per_category['Overall Homeless Veterans']
beds_per_category

## Calculating beds per homeless categories by Continuum of Cares

In [ ]:
coc_merge = pd.merge(coc_category, coc_beds)
coc_merge['Beds_Veteran'] = coc_merge['Dedicated Veteran Beds'] / coc_merge['Overall Homeless Veterans']
coc_merge['Beds_Youth'] = coc_merge['Dedicated Youth Beds'] / coc_merge['Overall Homeless Veterans']
coc_merge['Beds_Families'] = coc_merge['Total Beds for Households with Children'] / coc_merge['Overall Homeless Veterans']
coc_merge['Beds_Families_NoKids'] = coc_merge['Total Beds for Households without Children'] / coc_merge['Overall Homeless Veterans']
coc_merge['Peds_Per_Homeless'] = coc_merge['Total Year-Round Beds'] / coc_merge['Overall Homeless']
coc_merge['State'] = coc_merge['CoC Number'].str.extract(r'(^[A-Z]{2})')
coc_merge

## Exported all cleaned/aggregated dataframes to Excel

In [ ]:
#beds_pop.to_excel('beds_pop.xlsx', sheet_name = 'overall')
#beds_per_category.to_excel('beds_per_category.xlsx', sheet_name = 'beds_category')
#coc_merge.to_excel('coc_merge.xlsx', sheet_name = 'coc_data')
#year_perc.to_excel('year_percent.xlsx', sheet_name = 'percentages')

In [ ]:
#geo_coc = gpd.read_file('../Data/CoC_GIS_National_Boundary_2020/FY20_CoC_National_Bnd.gdb')
#geo_coc

In [ ]:
#geo_coc.to_excel('geo_coc.xlsx', sheet_name = 'geo')